In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


import scipy 
import scipy.optimize as so
import scipy.stats


from scipy import odr
from uncertainties import unumpy, ufloat

First of all we want to find the confidence interval for a given expectation value.

In the lecture we have seen for an upper bound, which is defined in way that for 10% of the possible values x we have x>x_m, it holds : $$ \int_{x_m}^{\infty} p(x) \,dx=0.10 $$ <br>
Similarly we found for a lower bound where 10% of the values x have x<x_m: $$ \int_{-\infty}^{x_m} p(x) \,dx=0.10 $$ <br>
If we want 10% outside a certain interval we have to calculate:

$$ \int_{x_2}^{\infty} p(x) \,dx=0.05 $$ <br>
$$ \int_{-\infty}^{x_1} p(x) \,dx=0.05 $$ <br>

That is what we want to calculate now for the poisson distribution for multiple $\lambda$

This means $$p(n,\lambda)=\lambda^n \cdot e^{-\lambda}\cdot \frac{1}{n!}$$

I) If we want 10% above n we search for the n value where the CDF hits 0.9

In [4]:
x=np.arange(0,100,1)

In [6]:
def upperbound(x,lamda):
    upperbound=np.argmin(np.abs(scipy.stats.poisson.cdf(x,lamda)-0.9))+1   #n for which the cdf comes closest to 0.9, the +1 is there because the python index starts at 0
    return upperbound

In [9]:
for i in range(1,13,1):
    print("lambda =", i, "upperbound=", upperbound(x,i))

lambda = 1 upperbound= 3
lambda = 2 upperbound= 4
lambda = 3 upperbound= 6
lambda = 4 upperbound= 7
lambda = 5 upperbound= 9
lambda = 6 upperbound= 10
lambda = 7 upperbound= 11
lambda = 8 upperbound= 12
lambda = 9 upperbound= 13
lambda = 10 upperbound= 15
lambda = 11 upperbound= 16
lambda = 12 upperbound= 17


Similarly we can search for the lower bounds by searching for the point where the cdf hits 0.1

In [10]:
def lowerbound(x,lamda):
    lowerbound=np.argmin(np.abs(scipy.stats.poisson.cdf(x,lamda)-0.1))+1   #n for which the cdf comes closest to 0.1, the +1 is there because the python index starts at 0
    return lowerbound

In [11]:
for i in range(1,13,1):
    print("lambda =", i, "lowerbound=", lowerbound(x,i))

lambda = 1 lowerbound= 1
lambda = 2 lowerbound= 1
lambda = 3 lowerbound= 1
lambda = 4 lowerbound= 2
lambda = 5 lowerbound= 3
lambda = 6 lowerbound= 3
lambda = 7 lowerbound= 4
lambda = 8 lowerbound= 5
lambda = 9 lowerbound= 6
lambda = 10 lowerbound= 7
lambda = 11 lowerbound= 7
lambda = 12 lowerbound= 8


Now we want to have 5% below our interval and 5% above our interval. So I can use a combination of both functions from above.

In [12]:
def upperbound2(x,lamda):
    upperbound=np.argmin(np.abs(scipy.stats.poisson.cdf(x,lamda)-0.95))+1   #n for which the cdf comes closest to 0.95, the +1 is there because the python index starts at 0
    return upperbound

def lowerbound2(x,lamda):
    lowerbound=np.argmin(np.abs(scipy.stats.poisson.cdf(x,lamda)-0.05))+1   #n for which the cdf comes closest to 0.05, the +1 is there because the python index starts at 0
    return lowerbound

In [14]:
for i in range(1,13,1):
    print("lambda =", i, "n=", lowerbound2(x,i), "n'=", upperbound2(x,i))

lambda = 1 n= 1 n'= 3
lambda = 2 n= 1 n'= 5
lambda = 3 n= 1 n'= 7
lambda = 4 n= 1 n'= 8
lambda = 5 n= 2 n'= 9
lambda = 6 n= 3 n'= 11
lambda = 7 n= 3 n'= 12
lambda = 8 n= 4 n'= 13
lambda = 9 n= 5 n'= 15
lambda = 10 n= 6 n'= 16
lambda = 11 n= 6 n'= 17
lambda = 12 n= 7 n'= 19


2) Now we want to find the confidence interval given a measured value. We want to find the confidence interval at 90% confidence level.

In the lecture we have seen for an upper bound $b$ of a poisson distribution $$\beta=\sum_{n=0}^{n_{obs}} \frac{b^{n}}{n!}\cdot e^{-b}$$ asnd for a lower bound $a$ of  a poisson distribution $$\alpha=1-\sum_{n=0}^{n_{obs}-1} \frac{a^{n}}{n!}\cdot e^{-a}$$ 

To get the upper bound at 90% confidence level we need $\beta=0.1$ and then we can solve the equation for b numerically.

In [17]:
def beta(b,n_obs):
    n_sum=0
    for n in range(0,n_obs+1):
        n_sub=b**n/scipy.special.factorial(n)*np.exp(-b)
        n_sum=n_sum+n_sub
        
    return n_sum

In [21]:
def upperbound_lamda(b,n_obs):
    return beta(b,n_obs)-0.1

In [30]:
for n_obs in np.arange(0,13,1):
    print("n=", n_obs, "lamda_upperbound=", scipy.optimize.root(upperbound_lamda,args=((n_obs)),x0=n_obs).x[0])

n= 0 lamda_upperbound= 2.3025850929940455
n= 1 lamda_upperbound= 3.889720169867429
n= 2 lamda_upperbound= 5.322320337834154
n= 3 lamda_upperbound= 6.680783068255857
n= 4 lamda_upperbound= 7.993589586052629
n= 5 lamda_upperbound= 9.27467389335162
n= 6 lamda_upperbound= 10.532072106498529
n= 7 lamda_upperbound= 11.770914461548056
n= 8 lamda_upperbound= 12.994711541318605
n= 9 lamda_upperbound= 14.205990292152816
n= 10 lamda_upperbound= 15.406641171976517
n= 11 lamda_upperbound= 16.598122144314086
n= 12 lamda_upperbound= 17.78158563596173


Similarly we can calculate the lower bound

In [34]:
def alpha(a,n_obs):
    n_sum=0
    for n in range(0,n_obs):
        n_sub=a**n/scipy.special.factorial(n)*np.exp(-a)
        n_sum=n_sum+n_sub
        
    return 1-n_sum

In [35]:
def lowerbound_lamda(a,n_obs):
    return alpha(a,n_obs)-0.1

In [36]:
for n_obs in np.arange(0,13,1):
    print("n=", n_obs, "lamda_lowerbound=", scipy.optimize.root(lowerbound_lamda,args=((n_obs)),x0=n_obs).x[0])

n= 0 lamda_lowerbound= 0.0
n= 1 lamda_lowerbound= 0.10536051565782632
n= 2 lamda_lowerbound= 0.5318116083896118
n= 3 lamda_lowerbound= 1.1020653282493216
n= 4 lamda_lowerbound= 1.7447695628249114
n= 5 lamda_lowerbound= 2.4325910259626653
n= 6 lamda_lowerbound= 3.151898029792162
n= 7 lamda_lowerbound= 3.894766804876186
n= 8 lamda_lowerbound= 4.656118176898029
n= 9 lamda_lowerbound= 5.432468058254662
n= 10 lamda_lowerbound= 6.221304605225033
n= 11 lamda_lowerbound= 7.0207465947109835
n= 12 lamda_lowerbound= 7.829342026256413


For central confidence intervals we need $\beta=0.05$ and $\alpha=0.05$. For this I just redefine my functions a little bit.

In [37]:
def upperbound_lamda(b,n_obs):
    return beta(b,n_obs)-0.05

def lowerbound_lamda(a,n_obs):
    return alpha(a,n_obs)-0.05


In [39]:
for n_obs in np.arange(0,13,1):
    print("n=", n_obs, "Central CIs at 90%=", [scipy.optimize.root(lowerbound_lamda,args=((n_obs)),x0=n_obs).x[0],scipy.optimize.root(upperbound_lamda,args=((n_obs)),x0=n_obs).x[0]])

n= 0 Central CIs at 90%= [0.0, 2.995732273553991]
n= 1 Central CIs at 90%= [0.05129329438755059, 4.743864518390578]
n= 2 Central CIs at 90%= [0.3553615106986622, 6.295793621871835]
n= 3 Central CIs at 90%= [0.8176914471639415, 7.753656527932705]
n= 4 Central CIs at 90%= [1.3663183967498316, 9.153519026637568]
n= 5 Central CIs at 90%= [1.9701495680595298, 10.513034908741531]
n= 6 Central CIs at 90%= [2.61301474419632, 11.842395652420288]
n= 7 Central CIs at 90%= [3.2853156918947026, 13.14811380243212]
n= 8 Central CIs at 90%= [3.98082278618951, 14.434649715196317]
n= 9 Central CIs at 90%= [4.69522754034449, 15.705216422115463]
n= 10 Central CIs at 90%= [5.425405697091292, 16.9622192357219]
n= 11 Central CIs at 90%= [6.169007289395323, 18.207514250903657]
n= 12 Central CIs at 90%= [6.924212513585108, 19.44256932991502]


2) Now we consider a poisson distribution which consists of signal and a background poisson distribution $$\frac{(\lambda_S+\lambda_B)^{n}}{n!}\cdot e^{-(\lambda_S+\lambda_B)}$$

Now wwe have observed n_obs=5 events. Furthermore we know $\lambda_B=1.8$ for the background. We now want to calculate the probability to see such a number of n_obs only from the background.

The probability to observe 5 or more evnts from the background can be calculated by $$p(n\geq n_{obs})=1-\sum_{n=0}^{n_{obs}-1} \frac{\lambda_{B}^{n}}{n!}\cdot e^{-\lambda_B}$$ 

In [41]:

lamdaB=1.8
n_obs=5

n_sum=0
for n in range(0,n_obs):
    n_sub=lamdaB**n/scipy.special.factorial(n)*np.exp(-lamdaB)
    n_sum=n_sum+n_sub
    
    
1-n_sum

0.03640666100108336

The probability to observe 5 or more events from the background is only 0.0364. But this is not enough to establish a signal at 3 sigma significance since for that we would need p=1-0.997=0.003 as probability.

2) Now we want to find an upper bound for $\lambda_S$ such that the probability to observe n_obs=5 or less reaches 5% when one assumes a poisson distribution with $\lambda=\lambda_{s_{max}}+\lambda_B$

This means we have $$\beta=0.05=\sum_{n=0}^{n_{obs}} \frac{(b+\lambda_{B})^{n}}{n!}\cdot e^{-(b+\lambda_B)}$$ with b our upper bound

I should be able to solve this in the same way as in exercise 1 part 2, so I did it in this way here. I am not sure how we are supposed to do the interval search here.

So I can define a function similarly to exercise 1

In [42]:
def beta2(b,n_obs):
    n_sum=0
    lamda_B=1.8
    for n in range(0,n_obs+1):
        n_sub=(b+lamda_B)**n/scipy.special.factorial(n)*np.exp(-(lamda_B+b))
        n_sum=n_sum+n_sub
        
    return n_sum

In [43]:
def upperbound_lamda_s(b,n_obs):
    return beta2(b,n_obs)-0.05

In [44]:
scipy.optimize.root(upperbound_lamda_s,args=((5)),x0=5).x[0]

8.713034908741532

The upper limit for $\lambda_S$ seems to be 8.713 to observe 5 events at a confidence level of 95%

3) Now I want to confirm this value by using Monte Carlo methods. I will draw a random number from a poisson distribution with $\lambda=\lambda_B+\lambda_{S_{max}}$ and then count the number of experiments i which this random number is less or equal to n_obs=5

In [76]:
# Note: lamda=lamda_B+lamda_s_max=8.713+1.8


count_array=np.zeros(0)
iterations=1000       #repeat the generation of 10000 values 1000 times

for j in range(iterations):
    
    count=0
    
    Lamda_array=np.random.poisson(8.713+1.8,10000)   #generate 10000 values and see how many are below n_obs=5
    for i in range(len(Lamda_array)):
        if Lamda_array[i]<=5:
            count+=1
        
    count_array=np.append([count_array],[count])    

print(np.mean(count_array))                                #mean number of how many random number where equal or below n_obs=5
print(np.mean(count_array/len(Lamda_array)))               #corresponding probability to get n_obs<=5

500.206
0.0500206


The probability is actually roughly 5%.